In [ ]:
!pip install transformers
!pip install openai-clip

Found existing installation: clip 0.2.0
Uninstalling clip-0.2.0:
  Would remove:
    /usr/local/bin/clip
    /usr/local/lib/python3.10/dist-packages/clip-0.2.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/clip/*
Proceed (Y/n)? y
  Successfully uninstalled clip-0.2.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.8 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368637 sha256=0d621b45acf57812b531df6572f3cfa63e82c3165d415b4d627021517406adc4
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip


In [ ]:
import random
import numpy as np

phrase_list = ['2d', 'pixel art', 'cave', 'scifi', 'side scrolling', 'chibi', 'waifu', 'space ship', 'desert', 'city', 'wasteland', 'mega structure', 'steal', 'stone', 'rock']

def prompt_generator(phrase_list, prompt_word_length=32):
    prompt = ''
    while len(prompt) < prompt_word_length:
        phrase = random.choice(phrase_list)
        if len(prompt) + len(phrase) + 1 <= prompt_word_length:
            prompt += phrase + ' '
        else:
            break
    return prompt.strip()

# example usage:
prompt=prompt_generator(phrase_list) 
print(prompt_generator(phrase_list))


wasteland 2d pixel art cave


In [ ]:
print(prompt)

scifi mega structure rock cave


In [ ]:
import torch
import clip
import numpy as np

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Tokenize and encode the prompt
prompt = "scifi mega structure rock cave"
tokens = clip.tokenize(prompt).to(device)
print("Size of tokens:", tokens.shape)
with torch.no_grad():
    text_features = model.encode_text(tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

# Convert the embedding to a numpy array
embedding = text_features.cpu().numpy()

print("Text embedding shape:",embedding.shape)  # should print (1, 512)




Size of tokens: torch.Size([1, 77])
Text embedding shape: (1, 512)


In [ ]:
import json
import torch
import clip

N=3

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def generate_prompts_with_embeddings(num_prompts):
    prompts = []
    for i in range(num_prompts):
        prompt = "scifi mega structure rock cave"
        tokens = clip.tokenize(prompt).to(device)
        text_features = model.encode_text(tokens).float()
        text_features /= text_features.norm(dim=-1, keepdim=True)
        prompt_data = {
            "prompt": prompt,
            "word_embedding": tokens[0].tolist(),
            "clip_embedding": text_features[0].tolist(),
        }
        prompts.append(prompt_data)
    
    with open("prompts_with_embeddings.json", "w") as f:
        json.dump(prompts, f)
generate_prompts_with_embeddings(N)
